# Подготовка данных для распознавания крокодилов и часов на изображениях

**Источник данных** - [Архив с изображениями](https://drive.google.com/file/d/1JbYmH50iRkMorFk0xNCnwC9xKiB60Mlq/view?usp=sharing), полученный по почте.

Данные разбиваются на три каталога:
- train (данные для обучения)
- val (данные для проверки)
- test (данные для тестирования)

В каждом каталоге создаются по два подкаталога, в соответсвии с названиями классов: crocodile и clock. 

Изображения переписваются из исходного каталога в новую структуру. По-умолчанию для обучения используется 70% изображений, для проверки - 15%, для тестрования также 15%. 

In [1]:
import shutil
import os

In [91]:
# Каталог с набором данных
data_dir = './clocks_crocodiles'
# Каталог с данными для обучения
train_dir = './data/train'
# Каталог с данными для проверки
val_dir = './data/val'
# Каталог с данными для тестирования
test_dir = './data/test'
# Часть набора данных для тестирования
test_data_portion = 0.15
# Часть набора данных для проверки
val_data_portion = 0.15
# Количество элементов данных в одном классе
nb_images = 500

Функция создания каталога с двумя подкаталогами по названию классов: clock и crocodile

In [82]:
def create_directory(dir_name):
    if os.path.exists(dir_name):
        shutil.rmtree(dir_name)
    os.makedirs(dir_name)
    os.makedirs(os.path.join(dir_name, "clock"))
    os.makedirs(os.path.join(dir_name, "crocodile"))    

Создание структуры каталогов для обучающего, проверочного и тестового набора данных

In [83]:
create_directory(train_dir)
create_directory(val_dir)
create_directory(test_dir)

Функция обзора директории, чтобы получить список файлов, из которых мы будем вытягивать изображения.

In [84]:
from collections import defaultdict
import os
def ls_scan_path(path='.'):
    """
    Get ENTRIES, DIRS, FILES and EXTS objects from the directory.
    ENTRIES - list of entries names
    DIRS    - list of tuples (directory_name, directory_size)
    FILES   - list of tuples (file_name, file_size)
    EXTS    - defaultdict with file extension strings as keys and dictionary as values
                EXTS[ext]['sizes'] - get sum of sizes of files with that extension.
                EXTS[ext]['files'] - a list of tuples (file_name, file_size) with that extension.
    """

    entries = [name for name in os.listdir(path)]

    files, dirs = [], []
    tree = lambda: defaultdict(tree)
    exts = tree()
    # Divide entries into files and directories, also filling exts dictionary
    for entry in entries:
        entry_path = os.path.join(path, entry)                  # Get entry path to use
        entry_size = os.path.getsize(entry_path)                # Get entry size
        entry_tuple = (entry, entry_size)                       # Create tuple for better use

        if os.path.isfile(entry_path):                          # If entry is a file
            files.append(entry_tuple)

            # Update extension dictionary
            filename = os.path.basename(entry_path)             # Get filename
            extension = os.path.splitext(entry_path)[1]         # get extension
            if filename[0] == '.':                              # For hidden files
                extension = 'Hidden'
            elif extension == '':                               # For files with no extension
                extension = 'None'
            exts[extension]['files'] = exts[extension].get('files', []) + [entry_tuple]
            exts[extension]['sizes'] = exts[extension].get('sizes', 0) + os.path.getsize(entry_path)

        elif os.path.isdir(entry_path):                         # If entry is a directory
            dirs.append(entry_tuple)
            extension = 'Directory'
            exts[extension]['files'] = exts[extension].get('files', []) + [entry_tuple]
            exts[extension]['sizes'] = exts[extension].get('sizes', 0) + os.path.getsize(entry_path)

    # Return everything
    return entries, dirs, files, exts

Используем эту функцию, чтобы переименовать все файлы из датасета в формат `class.number.jpg`. Для начала напишем ещё одну, чтобы быстро переименовывать папки.

In [85]:
def rename_all_in_dir(classname):
    class_dir = os.path.join(data_dir, classname)
    entries, dirs, files, exts = ls_scan_path(class_dir)

    for idx, file in enumerate(files):
        os.rename(os.path.join(class_dir, file[0]), 
                  os.path.join(class_dir, '{}.{}.png'.format(classname, idx)))

In [86]:
rename_all_in_dir('clock')
rename_all_in_dir('crocodile')

for i, j in enumerate([9,8,7]):
    print(i, j)

0 9
1 8
2 7


Функция копирования изображений в заданный каталог. Изображения часов и крокодилов копируются в отдельные подкаталоги

In [87]:
def copy_images(start_index, end_index, source_dir, dest_dir):
    clock_dir = os.path.join(data_dir, 'clock')
    croc_dir = os.path.join(data_dir, 'crocodile')
    
    for i in range(start_index, end_index):
        shutil.copy2(os.path.join(clock_dir, "clock." + str(i) + ".png"), 
                    os.path.join(dest_dir, "clock"))
        shutil.copy2(os.path.join(croc_dir, "crocodile." + str(i) + ".png"), 
                   os.path.join(dest_dir, "crocodile"))

Расчет индексов наборов данных для обучения, приверки и тестирования

In [88]:
start_val_data_idx = int(nb_images * (1 - val_data_portion - test_data_portion))
start_test_data_idx = int(nb_images * (1 - test_data_portion))
print(start_val_data_idx)
print(start_test_data_idx)               


350
425


In [89]:
!ls 

Samsung Tasks.pdf         clocks_crocodiles.rar     example.ipynb
clocks_crocodiles         data
clocks_crocodiles-1       dataset_preparation.ipynb


Копирование изображений

In [90]:
copy_images(0, start_val_data_idx, data_dir, train_dir)
copy_images(start_val_data_idx, start_test_data_idx, data_dir, val_dir)
copy_images(start_test_data_idx, nb_images, data_dir, test_dir)